# Работа с БД SQLite

In [3]:
import sqlite3
from prettytable import from_db_cursor

In [4]:
import os

# Временный каталог
TEMP_DIR = "temp"

if not os.path.exists(TEMP_DIR):
    os.mkdir(TEMP_DIR)

# Пример использования
os.path.join(TEMP_DIR, "temp_SQLite.db")

'temp\\temp_SQLite.db'

In [5]:
#db_name = os.path.join(TEMP_DIR, 'temp_SQLite.db')
db_name = ":memory:"

if os.path.isfile(db_name):
    os.remove(db_name)
    print(f"Файл [{db_name}] - удален")

In [6]:
db = sqlite3.connect(db_name)
cur = db.cursor()

In [7]:
#cur.execute("CREATE TABLE cities (name)")
#cur.execute("CREATE TABLE cities (name varchar(100))")
cur.execute("CREATE TABLE cities (id integer primary key AUTOINCREMENT NOT NULL, name varchar(100) unique)")
db.commit()

In [8]:
cur.execute("SELECT * FROM cities")
from_db_cursor(cur)

id,name


In [9]:
cur.execute("PRAGMA table_info('cities')")
from_db_cursor(cur)

cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,1,None,1
1,name,varchar(100),0,None,0


In [10]:
cur.execute("PRAGMA table_info('cities')")
t = from_db_cursor(cur)
print(t)

+-----+------+--------------+---------+------------+----+
| cid | name |     type     | notnull | dflt_value | pk |
+-----+------+--------------+---------+------------+----+
|  0  |  id  |   INTEGER    |    1    |    None    | 1  |
|  1  | name | varchar(100) |    0    |    None    | 0  |
+-----+------+--------------+---------+------------+----+


In [11]:
cities = "Москва, Воронеж, Калуга, Тула".split(', ')
cities

['Москва', 'Воронеж', 'Калуга', 'Тула']

In [12]:
cities_data = [(i,) for i in cities]
cities_data

[('Москва',), ('Воронеж',), ('Калуга',), ('Тула',)]

In [13]:
cur.execute("INSERT INTO cities (name) VALUES ('Ялта')")

In [14]:
cur.execute("SELECT * FROM cities")
from_db_cursor(cur)

id,name
1,Ялта


In [15]:
cur.executemany("INSERT INTO cities (name) VALUES (?)", [(i,) for i in cities])

cur.execute("SELECT * FROM cities")
from_db_cursor(cur)

id,name
1,Ялта
2,Москва
3,Воронеж
4,Калуга
5,Тула


In [16]:
cur.execute("SELECT * FROM cities")
print(*cur)

(1, 'Ялта') (2, 'Москва') (3, 'Воронеж') (4, 'Калуга') (5, 'Тула')


In [17]:
for line in cur.execute("SELECT * FROM cities"):
    print(line)

(1, 'Ялта')
(2, 'Москва')
(3, 'Воронеж')
(4, 'Калуга')
(5, 'Тула')


In [18]:
# стиль именованный - использование одного или нескольких ":<название>"
sql = "INSERT INTO cities (name) VALUES (:city)"

cur.execute(sql, {"city": "Belgrade"})

cur.execute("SELECT * FROM cities")
from_db_cursor(cur)

id,name
1,Ялта
2,Москва
3,Воронеж
4,Калуга
5,Тула
6,Belgrade


In [19]:
sql = "DELETE FROM cities WHERE id = ?"

cur.execute(sql, [6])

from_db_cursor(cur.execute("SELECT * FROM cities"))

id,name
1,Ялта
2,Москва
3,Воронеж
4,Калуга
5,Тула


In [20]:
sql = """
UPDATE cities
SET name = 'r. ' || name 
WHERE id = 1
    AND name NOT LIKE 'r. %'
"""
cur.execute(sql)

from_db_cursor(cur.execute("SELECT * FROM cities"))

id,name
1,r. Ялта
2,Москва
3,Воронеж
4,Калуга
5,Тула


## Извлечь данные из курсора

In [24]:
cur.execute("SELECT * FROM cities")
t = cur.fetchall()
print(t)
print(t[0])
print(t[:3])
print(t[::2])

[(1, 'r. Ялта'), (2, 'Москва'), (3, 'Воронеж'), (4, 'Калуга'), (5, 'Тула')]
(1, 'r. Ялта')
[(1, 'r. Ялта'), (2, 'Москва'), (3, 'Воронеж')]
[(1, 'r. Ялта'), (3, 'Воронеж'), (5, 'Тула')]


In [25]:
cur.description

(('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None))

In [27]:
cur.execute("PRAGMA table_info(`cities`)")
cur.fetchall()

[(0, 'id', 'INTEGER', 1, None, 1), (1, 'name', 'varchar(100)', 0, None, 0)]

In [29]:
cur.execute("PRAGMA table_info(`cities`)")
from_db_cursor(cur)

cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,1,None,1
1,name,varchar(100),0,None,0


In [30]:
cur.execute("SELECT * FROM cities").fetchone()

(1, 'r. Ялта')

In [31]:
cur.execute("SELECT * FROM cities").fetchmany(size=5)

[(1, 'r. Ялта'), (2, 'Москва'), (3, 'Воронеж'), (4, 'Калуга'), (5, 'Тула')]

In [34]:
cur.execute("SELECT * FROM cities").fetchall()[0:5]

[(1, 'r. Ялта'), (2, 'Москва'), (3, 'Воронеж'), (4, 'Калуга'), (5, 'Тула')]

## Работа с Join в БД SQLite

![](https://i.pinimg.com/originals/aa/ae/30/aaae30a99650820cb0802f9baf8f7d52.png)

In [21]:
db.close

<function Connection.close()>